In [1]:
import torch

In [2]:
class DenseLayer:

    def __init__(self,n_features, n_neurons):
        self.weights = torch.rand((n_features,n_neurons))
        self.bias = torch.rand(n_neurons)

    def forward(self,inputs):
        self.output = torch.matmul(inputs,self.weights) + self.bias

In [3]:
class Accuracy:

    def forward(self,y_pred,y_true):
        if y_pred.shape != y_true.shape:
            one_hot_notation = torch.zeros(y_pred.shape)
            one_hot_notation[range(len(y_pred)),y_true] = 1
        else:
            one_hot_notation = y_true
        correct_values = y_pred==one_hot_notation
        correct_values = correct_values * one_hot_notation
        self.output = torch.sum(correct_values) / len(y_pred)
        return self.output

In [4]:
def sigmoid(inputs):
    output = 1 / (1 + torch.exp(inputs*-1))
    return output

def relu(inputs):
    output = torch.max(inputs,torch.tensor(0))
    return output

In [5]:
class ClassificationModel:
    def __init__(self,num_of_features:int,num_of_classes:int):
        self.dense_layer = DenseLayer(num_of_features,4)
        self.output_layer = DenseLayer(4,num_of_classes)
        self.accuracy = Accuracy()
        self.errors = [float('inf')] * num_of_classes

    def forward_propagate(self,inputs):
        self.dense_layer.forward(inputs)
        self.activation1 = relu(self.dense_layer.output)
        self.output_layer.forward(self.activation1)
        self.activation2 = sigmoid(self.output_layer.output)
        self.inputs = inputs

    def loss_and_accuracy(self,output):
        self.true_value = output
        if self.activation2.shape != output.shape:
            one_hot_notation = torch.zeros(self.activation2.shape)
            one_hot_notation[range(len(self.activation2)),output] = 1
            self.true_value = one_hot_notation

        loss = ((self.true_value - self.output_layer.output) ** 2) / 2
        loss = torch.mean(loss)
        accuracy = self.accuracy.forward(self.output_layer.output,self.true_value)
        return loss, accuracy


    def back_prop(self,lr):
        errors = -(self.true_value - self.output_layer.output) # d(y-output)/d(output)
        avg_errors = torch.sum(errors,keepdims=True,dim=0) / len(errors)
        avg_errors = torch.squeeze(avg_errors)
        avg_outputs = torch.sum(self.activation2,keepdims=True,dim=0) / len(self.activation2)
        avg_outputs = torch.squeeze(avg_outputs)
        avg_act1_outputs = torch.sum(self.activation1,keepdims=True,dim=0) / len(self.activation1)
        avg_act1_outputs = torch.squeeze(avg_act1_outputs)
        avg_inputs = torch.sum(self.inputs,keepdims=True,dim=0) / len(self.inputs)
        avg_inputs = torch.squeeze(avg_inputs)
        avg_layer1_output = torch.sum(self.dense_layer.output,keepdims=True,dim=0) / len(self.dense_layer.output)
        avg_layer1_output = torch.squeeze(avg_layer1_output)
        back = [None] * 4
        for j in range(4):
            back[j] = torch.tensor(lr) * avg_errors[j] * (avg_outputs[j] * (1-avg_outputs[j]))

        for i in range(4):
            self.output_layer.bias[i] -= back[i]
            for j in range(4):
                self.output_layer.weights[i][j] -= back[j] * avg_act1_outputs[i]


        for j in range(4):
            for k in range(4):
                self.dense_layer.bias[j] -= back[j] * self.output_layer.weights[j][k] * (1 if avg_layer1_output[j]>0 else 0)
                for i in range(2):
                    self.dense_layer.weights[i][j] -= back[k] * self.output_layer.weights[j][k] * avg_inputs[i] * (1 if avg_layer1_output[j]>0 else 0)


In [7]:
model = ClassificationModel(2,4)
x = torch.tensor([[1,2],[3,4],[4,5]],dtype=torch.float)
y = torch.tensor([1,1,0])
model.forward_propagate(x)
model.loss_and_accuracy(y)

loss = 0.1
error = float('inf')
iterations = 0
while loss < error:
    iterations += 1
    model.forward_propagate(x)
    model.back_prop(0.01)
    error,acc = model.loss_and_accuracy(y)

print("iterations:",iterations)
print("final output:",model.output_layer.output)
print("true vaues:",model.true_value)
print("accuracy:",acc)
print("final error:",error)


iterations: 98
final output: tensor([[0.1972, 0.4004, 0.2947, 0.0636],
        [0.2549, 0.6210, 0.4795, 0.1627],
        [0.2925, 0.7313, 0.5880, 0.2223]])
true vaues: tensor([[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.]])
accuracy: tensor(0.)
final error: tensor(0.0994)
